In [17]:
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
import random

# Фиксация random seed для воспроизводимости
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)


# Игнорировать все предупреждения
warnings.filterwarnings('ignore')

# Загрузка данных
file_path = "Meteo1-2023-5min_dropped.csv"
df = pd.read_csv(file_path, delimiter=",")

# Преобразование времени в datetime и установка индекса
df["time_YYMMDD_HHMMSS"] = pd.to_datetime(
    df["time_YYMMDD_HHMMSS"], format="%Y-%m-%d %H:%M:%S"
)
df.set_index("time_YYMMDD_HHMMSS", inplace=True)

# Добавление новых признаков
df["month"] = df.index.month
df["day_of_month"] = df.index.day
df["hour"] = df.index.hour

# Просмотр первых строк датафрейма с новыми признаками
print(df.head())

                     WindSpeedMax  WindDirection  AirTemperature  AirPressure  \
time_YYMMDD_HHMMSS                                                              
2023-01-01 00:00:00           5.4      285.00000        4.900000    1028.3000   
2023-01-01 00:15:00           5.6      295.16653        4.985714    1028.4001   
2023-01-01 00:30:00           4.8      306.24356        5.000000    1028.5000   
2023-01-01 00:45:00           5.9      298.47113        4.900001    1028.4601   
2023-01-01 01:00:00           6.0      299.20508        4.566667    1028.4001   

                     AirHumidity  WindSpeed  month  day_of_month  hour  
time_YYMMDD_HHMMSS                                                      
2023-01-01 00:00:00    88.900000   5.100000      1             1     0  
2023-01-01 00:15:00    88.871440   4.614286      1             1     0  
2023-01-01 00:30:00    88.392870   3.771429      1             1     0  
2023-01-01 00:45:00    90.213350   4.533333      1             1   

In [18]:
from utils import cyclical_feature_encoding

df_transformed = cyclical_feature_encoding(
    df, ["WindDirection", "month", "day_of_month", "hour"]
)
print(df_transformed.head())
print(df_transformed.tail())

                     WindSpeedMax  AirTemperature  AirPressure  AirHumidity  \
time_YYMMDD_HHMMSS                                                            
2023-01-01 00:00:00           5.4        4.900000    1028.3000    88.900000   
2023-01-01 00:15:00           5.6        4.985714    1028.4001    88.871440   
2023-01-01 00:30:00           4.8        5.000000    1028.5000    88.392870   
2023-01-01 00:45:00           5.9        4.900001    1028.4601    90.213350   
2023-01-01 01:00:00           6.0        4.566667    1028.4001    89.859985   

                     WindSpeed  WindDirection_sin  WindDirection_cos  \
time_YYMMDD_HHMMSS                                                     
2023-01-01 00:00:00   5.100000          -0.965883           0.258977   
2023-01-01 00:15:00   4.614286          -0.905004           0.425404   
2023-01-01 00:30:00   3.771429          -0.806407           0.591361   
2023-01-01 00:45:00   4.533333          -0.878976           0.476866   
2023-01-01 01:

In [19]:
from utils import generate_time_series_features, calculate_horizons

# Используем параметры, которые дали лучший результат
lookback_hours = 12
forecast_hours = 3
lag_step = 4
rolling_window_min = 3
rolling_window_max = 15
expanding_window_min = 3
expanding_window_max = 15

target_variable = "WindSpeed"

# Вычисление горизонта прогноза и лагов
forecast_horizon, lookback_horizon = calculate_horizons(
    df_transformed, forecast_hours, lookback_hours
)

lags = list(range(1, lookback_horizon, lag_step))
rolling_window_sizes = list(
    range(rolling_window_min, min(rolling_window_max, lookback_horizon), lag_step)
)
expanding_window_sizes = list(
    range(expanding_window_min, min(expanding_window_max, lookback_horizon), lag_step)
)

# Генерация признаков
df_transformed = generate_time_series_features(
    df_transformed,
    columns=[
        "WindSpeedMax",
        "AirTemperature",
        "AirPressure",
        "AirHumidity",
        "WindSpeed",
        "WindDirection_sin",
        "WindDirection_cos",
    ],
    lags=lags,
    rolling_window_sizes=rolling_window_sizes,
    expanding_window_sizes=expanding_window_sizes,
)

# Display the transformed DataFrame
df_transformed.head()


,WindSpeedMax,AirTemperature,AirPressure,AirHumidity,WindSpeed,WindDirection_sin,WindDirection_cos,month_sin,month_cos,day_of_month_sin,...,WindDirection_cos_lag_13,WindDirection_cos_lag_9,WindDirection_cos_lag_5,WindDirection_cos_lag_1,WindDirection_cos_rolling_mean_3,WindDirection_cos_rolling_mean_7,WindDirection_cos_rolling_mean_11,WindDirection_cos_cum_mean_3,WindDirection_cos_cum_mean_7,WindDirection_cos_cum_mean_11
time_YYMMDD_HHMMSS,,,,,,,,,,,,,,,,,,,,,
2023-01-01 11:15:00,2.4,14.153335,1027.8868,57.426670,1.733333,-0.574761,-0.818321,0.5,0.866025,0.201299,...,0.265990,0.365512,0.250449,-0.592528,-0.518422,-0.138575,0.032546,0.326125,0.326125,0.326125
2023-01-01 11:30:00,2.1,14.364284,1027.9001,55.571430,1.600000,0.289938,-0.957045,0.5,0.866025,0.201299,...,0.271629,0.416031,0.453822,-0.818321,-0.789298,-0.298194,-0.085670,0.298823,0.298823,0.298823
2023-01-01 11:45:00,3.1,14.785715,1027.7999,53.957146,2.071429,0.242793,-0.970078,0.5,0.866025,0.201299,...,0.341415,0.203157,-0.279316,-0.957045,-0.915148,-0.472555,-0.207087,0.272388,0.272388,0.272388
2023-01-01 12:00:00,1.9,13.528570,1027.7999,59.664288,1.242857,0.999980,-0.006325,0.5,0.866025,0.201299,...,0.343324,0.160289,-0.144418,-0.970078,-0.644483,-0.538290,-0.245483,0.266700,0.266700,0.266700
2023-01-01 12:15:00,3.4,12.000000,1027.7001,70.471430,2.750000,0.782221,0.623002,0.5,0.866025,0.201299,...,0.365512,0.250449,-0.592528,-0.006325,-0.117800,-0.409388,-0.207315,0.273826,0.273826,0.273826


In [20]:
from sklearn.model_selection import train_test_split
from utils import prepare_ml_data_from_features

# Разделение данных на обучающую и тестовую выборки
df_train, df_test = train_test_split(df_transformed, test_size=0.1, random_state=42, shuffle=False)

# Подготовка данных для машинного обучения
X_train, y_train = prepare_ml_data_from_features(df_train, target_variable=target_variable, forecast_horizon=forecast_horizon)
X_test, y_test = prepare_ml_data_from_features(df_test, target_variable=target_variable, forecast_horizon=forecast_horizon)

# Вывод размеров данных
print(f"Размеры обучающего набора X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размеры тестового набора X_test: {X_test.shape}, y_test: {y_test.shape}")

Размеры обучающего набора X_train: (31484, 139), y_train: (31484, 12)
Размеры тестового набора X_test: (3488, 139), y_test: (3488, 12)


In [21]:
from sklearn.model_selection import train_test_split
from utils import prepare_ml_data_from_features

# Разделение данных на обучающую и тестовую выборки
df_train, df_test = train_test_split(df_transformed, test_size=0.1, random_state=42, shuffle=False)

# Подготовка данных для машинного обучения
X_train, y_train = prepare_ml_data_from_features(df_train, target_variable=target_variable, forecast_horizon=forecast_horizon)
X_test, y_test = prepare_ml_data_from_features(df_test, target_variable=target_variable, forecast_horizon=forecast_horizon)

# Вывод размеров данных
print(f"Размеры обучающего набора X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размеры тестового набора X_test: {X_test.shape}, y_test: {y_test.shape}")


Размеры обучающего набора X_train: (31484, 139), y_train: (31484, 12)
Размеры тестового набора X_test: (3488, 139), y_test: (3488, 12)


In [22]:
import plotly.graph_objs as go
import plotly.subplots as sp
import numpy as np

def plot_sample_with_features(X, y, target_variable, columns, y_pred_values=None):
    """
    Функция для визуализации одного случайного примера из данных X и y с использованием Plotly.
    
    :param X: Матрица признаков (включая лаги целевой переменной)
    :param y: Целевая переменная (например, ветер на 12 шагов вперед)
    :param target_variable: Название целевой переменной
    :param columns: Список всех колонок, которые будут визуализироваться
    :param y_pred_values: Прогнозируемые значения (если есть)
    """
    # Выбор случайного индекса
    idx = np.random.randint(0, len(X))

    # Фильтрация лагов для каждой колонки в columns и сортировка по убыванию
    lagged_features = {col: [col_name for col_name in X.columns if f"{col}_lag" in col_name] for col in columns}

    # Создание субплотов
    num_features = len(columns)
    fig = sp.make_subplots(rows=num_features, cols=1, shared_xaxes=False, vertical_spacing=0.05)

    for i, column in enumerate(columns):
        # Данные по лагам для текущей колонки и текущее значение
        x_lags = X[lagged_features[column] + [column]].iloc[idx].values

        # Добавление данных на график для текущего признака
        fig.add_trace(go.Scatter(x=list(range(len(x_lags))), y=x_lags,
                                 mode='lines+markers', name=f'{column} (lags and current)',
                                 line=dict(color='green'), showlegend=(i == 0)), row=i+1, col=1)

        if column == target_variable:
            # Значения y для прогноза, только для целевой переменной
            y_values = y[idx]

            fig.add_trace(go.Scatter(x=list(range(len(x_lags), len(x_lags) + len(y_values))), y=y_values,
                                     mode='lines+markers', name=f'{column} (Future)',
                                     line=dict(color='red'), showlegend=(i == 0)), row=i+1, col=1)

            if y_pred_values is not None:
                fig.add_trace(go.Scatter(x=list(range(len(x_lags), len(x_lags) + len(y_pred_values[idx]))), y=y_pred_values[idx],
                                         mode='lines+markers', name=f'{column} (Predicted)',
                                         line=dict(color='blue'), showlegend=(i == 0)), row=i+1, col=1)

        # Вертикальная линия для разделения лагов и прогнозов
        fig.add_vline(x=len(x_lags)-1, line=dict(color='gray', dash='dash'), row=i+1, col=1)

    # Обновление разметки и отображение графика
    fig.update_layout(height=num_features * 400, width=1500, title_text="Sample Visualization with Features")
    fig.show()

# Пример использования
plot_sample_with_features(X_train, y_train, target_variable='WindSpeed', columns=[
        "WindSpeedMax",
        "AirTemperature",
        "AirPressure",
        "AirHumidity",
        "WindSpeed",
        "WindDirection_sin",
        "WindDirection_cos",
    ])

In [23]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from utils import calculate_metrics


# Масштабирование данных X
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Масштабирование данных y
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Формат данных
input_shape = (X_train_scaled.shape[1],)  # Входной вектор признаков
forecast_horizon = y_train_scaled.shape[1]  # Количество шагов вперед

# Создание модели с оптимальными параметрами
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=input_shape))  # Уменьшение нейронов на основе оптимального результата
model.add(Dense(100, activation='relu'))  # Оптимальные 5 слоев
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))  # Оптимальный последний слой с 50 нейронами
model.add(Dense(forecast_horizon))  # Прогнозируем N шагов вперед

# Компиляция модели с заданным learning rate
model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse')

# Настройка коллбека для сохранения лучших весов модели по валидационной потере
checkpoint = ModelCheckpoint('best_model.weights.h5', monitor='val_loss', save_best_only=True, mode='min', save_weights_only=True)

# Настройка коллбека для динамического изменения learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Обучение модели с использованием reduce_lr
model.fit(X_train_scaled, y_train_scaled, epochs=150, batch_size=256, validation_split=0.2, callbacks=[checkpoint, reduce_lr])

# Загрузка лучших сохраненных весов
model.load_weights('best_model.weights.h5')

# Прогнозирование
y_pred_scaled = model.predict(X_test_scaled)

# Обратное масштабирование y_pred
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Вывод метрик
print(calculate_metrics(y_test, y_pred, y_train))

Epoch 1/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.9853 - val_loss: 0.9383 - learning_rate: 5.0000e-05
Epoch 2/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7625 - val_loss: 0.6407 - learning_rate: 5.0000e-05
Epoch 3/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5585 - val_loss: 0.5264 - learning_rate: 5.0000e-05
Epoch 4/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4883 - val_loss: 0.5052 - learning_rate: 5.0000e-05
Epoch 5/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4654 - val_loss: 0.5008 - learning_rate: 5.0000e-05
Epoch 6/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4525 - val_loss: 0.5012 - learning_rate: 5.0000e-05
Epoch 7/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4431 - val_loss: 0.5042 - learning_rate: 5.0000e-05
Epoch 8/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4355 - val_loss: 0.5078 - learning_rate: 5.0000e-05
Epoch 9/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4289 - val_loss: 0.5106 - learning_

In [24]:
y_pred.shape

(3488, 12)

In [44]:
plot_sample_with_features(X_test, y_test, target_variable='WindSpeed', columns=[
        "WindSpeed"], y_pred_values = y_pred
    )

In [26]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test[:, 0]))),
    y=y_test[:, 0],
    mode='lines',
    name='Actual',
    line=dict(color='blue')
))

# Add predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_pred[:, 0]))),
    y=y_pred[:, 0],
    mode='lines',
    name='Predicted',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values',
    xaxis_title='Time Steps',
    yaxis_title='Value',
    legend=dict(x=0, y=1),
    width=1800,
    height=600,
    showlegend=True
)

# Show the plot
fig.show()


In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot figure with 12 rows and 1 column
fig = make_subplots(rows=12, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Iterate over each forecast step and add a trace for actual and predicted values
for i in range(12):
    fig.add_trace(go.Scatter(
        x=list(range(len(y_test[:, i]))),
        y=y_test[:, i],
        mode='lines',
        name=f'Actual (Step {i+1})',
        line=dict(color='blue')
    ), row=i+1, col=1)
    
    fig.add_trace(go.Scatter(
        x=list(range(len(y_pred[:, i]))),
        y=y_pred[:, i],
        mode='lines',
        name=f'Predicted (Step {i+1})',
        line=dict(color='red')
    ), row=i+1, col=1)

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values for Each Forecast Step',
    height=3000,  # Increase height to fit all subplots
    showlegend=False  # Disable legend to avoid repetition
)

# Show the plot
fig.show()


In [28]:
from utils import stitch_forecasts

# Применяем функцию к y_pred и y_test
y_pred_stitched = stitch_forecasts(y_pred)
y_test_stitched = stitch_forecasts(y_test)

y_pred_stitched.shape

(3492,)

In [45]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test_stitched))),
    y=y_test_stitched,
    mode='lines',
    name='Actual',
    line=dict(color='blue')
))

# Add predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_pred_stitched))),
    y=y_pred_stitched,
    mode='lines',
    name='Predicted',
    line=dict(color='red')
))

# Add vertical lines every 12 values
for i in range(0, len(y_test_stitched), 12):
    fig.add_shape(
        type='line',
        x0=i,
        x1=i,
        y0=min(min(y_test_stitched), min(y_pred_stitched)),
        y1=max(max(y_test_stitched), max(y_pred_stitched)),
        line=dict(color='rgba(0, 0, 0, 0.5)', width=1)
    )

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values',
    xaxis_title='Time Steps',
    yaxis_title='Value',
    legend=dict(x=0, y=1),
    width=1800,
    height=600,
    showlegend=True
)

# Show the plot
fig.show()
